# recursive <acronym title="Common Table Expression">CTE</acronym>s

When you are  tempted to open a `CURSOR` 👴, consider <acronym title="Common Table Expression">CTE</acronym>s instead [📖 [docs](https://learn.microsoft.com/en-us/sql/t-sql/queries/with-common-table-expression-transact-sql?view=sql-server-ver16)]. The imperative way to search through Microsoft SQL Server data ultimately depends on `CURSOR` \[📖 [docs](https://learn.microsoft.com/en-us/sql/relational-databases/cursors?view=sql-server-ver16) \] and probably a bunch of temp tables.

The modern, set-based way to consider is a design featuring <acronym title="Common Table Expression">CTE</acronym>s. See “[SQL SERVER – Replacing a Cursor with a Common Table Expression](https://blog.sqlauthority.com/2023/07/03/sql-server-replacing-a-cursor-with-a-common-table-expression/).”

The most compelling reason to move to <acronym title="Common Table Expression">CTE</acronym>s is for elegance around recursion. Consider the following set of employees:

In [1]:
USE pubs

CREATE TABLE #Employees
    (
        [ID] INT
    ,   [NAME] VARCHAR(32)
    ,   [ROLE] VARCHAR(32)
    ,   [MANAGER_ID] INT
    )

INSERT INTO #Employees
VALUES
    ( 7369, 'SMITH', 'CLERK', 7902 )
,   ( 7499, 'ALLEN', 'SALESMAN', 7698 )
,   ( 7521, 'WARD', 'SALESMAN', 7698 )
,   ( 7566, 'JONES', 'MANAGER', 7839 )
,   ( 7654, 'MARTIN', 'SALESMAN', 7698 )
,   ( 7698, 'BLAKE', 'MANAGER', 7839 )
,   ( 7782, 'CLARK', 'MANAGER', 7839 )
,   ( 7788, 'SCOTT', 'ANALYST', 7566 )
,   ( 7839, 'KING', 'PRESIDENT', NULL )
,   ( 7844, 'TURNER', 'SALESMAN', 7698 )
,   ( 7876, 'ADAMS', 'CLERK', 7788 )
,   ( 7900, 'JAMES', 'CLERK', 7698 )
,   ( 7902, 'FORD', 'ANALYST', 7566 )
,   ( 7934, 'MILLER', 'CLERK', 7782 )

SELECT * FROM #Employees

(14 rows affected)

(14 rows affected)

Total execution time: 00:00:00.035

ID,NAME,ROLE,MANAGER_ID
7369,SMITH,CLERK,7902
7499,ALLEN,SALESMAN,7698
7521,WARD,SALESMAN,7698
7566,JONES,MANAGER,7839
7654,MARTIN,SALESMAN,7698
7698,BLAKE,MANAGER,7839
7782,CLARK,MANAGER,7839
7788,SCOTT,ANALYST,7566
7839,KING,PRESIDENT,NULL
7844,TURNER,SALESMAN,7698


We can report on who is managed by whom with recursion:

In [2]:
;WITH EMPLOYEE_CTE AS (
    -- seed record:
    SELECT
        [ID]
    ,   [NAME]
    ,   [ROLE]
    ,   [MANAGER_ID]
    ,   [MANAGER_NAME] = CAST('N/A' AS VARCHAR(32))
    FROM #Employees
    WHERE [MANAGER_ID] IS NULL

    UNION ALL

    -- recursive statement:
    SELECT
        emps.[ID]
    ,   emps.[NAME]
    ,   emps.[ROLE]
    ,   emps.[MANAGER_ID]
    ,   [MANAGER_NAME] = cte.[NAME]
    FROM
        #Employees emps
        --constrain and recur:
        INNER JOIN EMPLOYEE_CTE cte
            ON cte.[ID] = emps.[MANAGER_ID]
)
SELECT * FROM EMPLOYEE_CTE

(14 rows affected)

Total execution time: 00:00:00.003

ID,NAME,ROLE,MANAGER_ID,MANAGER_NAME
7839,KING,PRESIDENT,NULL,N/A
7566,JONES,MANAGER,7839,KING
7698,BLAKE,MANAGER,7839,KING
7782,CLARK,MANAGER,7839,KING
7934,MILLER,CLERK,7782,CLARK
7499,ALLEN,SALESMAN,7698,BLAKE
7521,WARD,SALESMAN,7698,BLAKE
7654,MARTIN,SALESMAN,7698,BLAKE
7844,TURNER,SALESMAN,7698,BLAKE
7900,JAMES,CLERK,7698,BLAKE


There are two important things to remember:
1. `EMPLOYEE_CTE` is filling with every iteration.
2. The `INNER JOIN` constrains recursion _and_ drives recursion.

The inner join on `cte.[ID]` _up_ to `emps.[MANAGER_ID]` recognizes the hierarchical relation between `[ID]` and `[MANAGER_ID]`.

Also, to report on the hierarchy of a _specific_ manager change the `WHERE [MANAGER_ID] IS NULL` constraint to  `WHERE [ID] = <ID of employee>`.

In [3]:
DROP TABLE #Employees

Commands completed successfully.

Total execution time: 00:00:00.003

## recursive <acronym title="Common Table Expression">CTE</acronym> as a counter

Previously was the assertion that `EMPLOYEE_CTE` is filling with every _iteration_. We can explicitly demonstrate this iterative quality of <acronym title="Common Table Expression">CTE</acronym>s with the following:

In [4]:
WITH COUNTER_CTE AS (
    -- seed record:
    SELECT n = 0

    UNION ALL

    -- recursive statement:
    SELECT n + 1     -- increment projection
    FROM COUNTER_CTE -- recursive `FROM` clause
    WHERE n < 10     -- limit on recursion
)
SELECT n FROM COUNTER_CTE

(11 rows affected)

Total execution time: 00:00:00.002

n
0
1
2
3
4
5
6
7
8
9


The elements of <acronym title="Common Table Expression">CTE</acronym> recursion include:

- an “anchor” ⚓ or seed 🌱 record
- a `UNION` statement that allows duplicates
- a recursive statement

Note that our first <acronym title="Common Table Expression">CTE</acronym> example achieved recursion by a `JOIN` and the example above would run “forever” (until stack overflow) without the `WHERE` constraint.

For more detail, watch the following YouTube video:

<figure> 
    <a href="https://www.youtube.com/watch?v=IBSmPZFR9Pg"> 
        <img alt="SQL Recursion with CTE Part 1 | Quick Tips Ep59" src="https://img.youtube.com/vi/IBSmPZFR9Pg/maxresdefault.jpg" width="480" /> 
    </a> 
    <p><small>SQL Recursion with CTE Part 1 | Quick Tips Ep59</small></p> 
</figure> 


## <!-- -->

🐙🐱[BryanWilhite](https://github.com/BryanWilhite)